#### Этот Ноутбук являетсся Примером/Шаблоном к этому заданию (Baseline) и не служит готовым решением!     
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат. 

# Import

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
#!pip freeze > requirements.txt

# DATA

In [4]:
DATA_DIR = './'
data = pd.read_csv(DATA_DIR+'stud_math.csv.xls')

FileNotFoundError: [Errno 2] File ./stud_math.csv.xls does not exist: './stud_math.csv.xls'

In [5]:
data.info()

NameError: name 'data' is not defined

In [6]:
data.head(10)

NameError: name 'data' is not defined

### Первоначальное описание колонок:
1 **school** — аббревиатура школы, в которой учится ученик;

2 **sex** — пол ученика (**'F'** - женский, **'M'** - мужской);

3 **age** — возраст ученика (от **15** до **22**);

4 **address** — тип адреса ученика (**'U'** - городской, **'R'** - за городом);

5 **famsize** — размер семьи(**'LE3'** <= 3, **'GT3'** >3);

6 **p_status** — статус совместного жилья родителей (**'T'** - живут вместе **'A'** - раздельно);

7 **m_edu** — образование матери (**0** - нет, **1** - 4 класса, **2** - 5-9 классы,
    **3** - среднее специальное или 11 классов, **4** - высшее);

8 **f_edu** — образование отца (**0** - нет, **1** - 4 класса, **2** - 5-9 классы, 
    **3** - среднее специальное или 11 классов, **4** - высшее);

9 **m_job** — работа матери (**'teacher'** - учитель, **'health'** - сфера здравоохранения, 
    **services'** - гос служба, **'at_home'** - не работает, **'other'** - другое);

10 **f_job** — работа отца (**'teacher'** - учитель, **'health'** - сфера здравоохранения, 
    **'services'** - гос служба, **'at_home'** - не работает, **'other'** - другое);

11 **reason** — причина выбора школы (**'home'** - близость к дому, **'reputation'** - репутация школы, 
    **'course'** - образовательная программа, **'other'** - другое);

12 **guardian** — опекун (**'mother'** - мать, **'father'** - отец, **'other'** - другое);

13 **traveltime** — время в пути до школы (**1** - <15 мин., **2** - 15-30 мин., 
    **3** - 30-60 мин., **4** - >60 мин.);

14 **studytime** — время на учёбу помимо школы в неделю (**1** - <2 часов, **2** - 2-5 часов, 
    **3** - 5-10 часов, **4** - >10 часов);

15 **failures** — количество внеучебных неудач (**n**, если 1<=n<3, иначе **4**);

16 **schoolsup** — дополнительная образовательная поддержка (**yes** или **no**);

17 **famsup** — семейная образовательная поддержка (**yes** или **no**);

18 **paid** — дополнительные платные занятия по математике (**yes** или **no**);

19 **activities** — дополнительные внеучебные занятия (**yes** или **no**);

20 **nursery** — посещал детский сад (**yes** или **no**);

**studytime_granular** - неизвестная колонка, необходимо понять, с чем она коррелирует

21 **higher** — хочет получить высшее образование (**yes** или **no**);

22 **internet** — наличие интернета дома (**yes** или **no**);

23 **romantic** — в романтических отношениях (**yes** или **no**);

24 **famrel** — семейные отношения (от **1** - очень плохо до **5** - очень хорошо);

25 **freetime** — свободное время после школы (от **1** - очень мало до **5** - очень мого);

26 **goout** — проведение времени с друзьями (от **1** - очень мало до **5** - очень много);

27 **health_level** — текущее состояние здоровья (от **1** - очень плохо до **5** - очень хорошо);

28 **absences** — количество пропущенных занятий;

29 **score** — баллы по госэкзамену по математике (целевой параметр).

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# 1. EDA  
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками. В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой. 
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

## Анализ признаков
* Строим графики
* Проводим корреляционный анализ
* Находим дублирующиеся признаки
* Находим выбросы
* Cмотрим распределение целевой переменной
* Распределение целевой переменной относительно признаков
* Находим Возможные закономерности и взаимосвязи

In [7]:
# ...

## Выводы

In [8]:
# ...

# Bonus!
Если Хотите построить модель на основе отобранных признаков то можете продолжить далее. Этот материал забегает немного вперед, но зато позволит лучше понять зачем этот EDA вообще нужно было делать)

# 2. Подготовка данных для модели
Для того чтоб наша модель могла работать с нашими признаки они должны быть в цифровом формате понятной машине (в форматах int или float и без None-значений).

### 2.1. Обработка категориальных признаков

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [9]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
# Тут я возьму в обработку все object признаки.
object_columns = [s for s in data.columns if data[s].dtypes == 'object']

data = pd.get_dummies(data, columns=object_columns, dummy_na=True)

# определите какие признаки у вас пойдут в OH енкодинг.

NameError: name 'data' is not defined

In [10]:
data.head(5)

NameError: name 'data' is not defined

In [11]:
data.sample(5)

NameError: name 'data' is not defined

### 2.2. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [12]:
# в этом примере я заполняю пропуски median-ой, попробуйте другие подходы
nan_columns = list(data.columns[data.isnull().sum() > 0])
if nan_columns:
    for nan_column in nan_columns:
        data[nan_column+'_isNAN'] = pd.isna(data[nan_column]).astype('uint8')
    data.fillna(data.median(), inplace=True)

NameError: name 'data' is not defined

### Итого

In [13]:
data.head(5)

NameError: name 'data' is not defined

In [14]:
data.info()

NameError: name 'data' is not defined

Пропуск нет, все признаки в цифровом виде - данные готовы к работе с моделью

### 2.3 Split Data
чтоб сделать оценку на сколько хороша наша модель, сделаем разбивку данных на обучение и тест

In [15]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(data.drop(['score'],axis=1), 
                                                     data.score, 
                                                     test_size=0.2, 
                                                     random_state=RANDOM_SEED)

NameError: name 'data' is not defined

In [16]:
# проверяем
data.shape, X_train.shape, X_test.shape

NameError: name 'data' is not defined

# 3. Model 

In [17]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [18]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [19]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

NameError: name 'X_train' is not defined

In [20]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
# Чем меньше - тем лучше
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

NameError: name 'y_test' is not defined

# А зачем тут EDA?
Большинство последних соревнований по ML было выиграно благодаря глубокому понимаюнию самих данных и Подробному EDA! Чем лучше вы понимание сами признаки и что за ними стоит, тем проще вам будет ими манипулировать и генерировать новые сильные признаки.

Теперь на основе EDA попробуем это сделать сами:
* Удалите лишние по вашему мнению признаки
* Попробуйте по другому обработать пропуски и выбросы
* Попробуйте извлечь дополнительные признаки 
* Посмотрите какие могут быть глубокие взаимосвязи между признаками и таргетом, и как можно помочь модели их увидеть. 

Посмотрите как это влияет на результат модели...    
Поделитесь своими результатами и идеями с коллегами в Slacke